In [1]:
import torch
import torch.nn as nn
import altair as alt
import pandas as pd
import numpy as np
import warnings
from pathlib import Path
warnings.filterwarnings("ignore")

In [2]:
# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
torch.cuda.empty_cache()


Using device: cuda


In [45]:
from train import DataSetLoader
from config import get_config
from transformer import Transformer
from transformer import TransformerBuilder

In [ ]:
config = get_config()
train_dataloader, validation_dataloader, test_dataloader, vocab_source, vocab_target = DataSetLoader.get_dataset(config)

In [47]:
model = DataSetLoader.get_model(config, vocab_source.get_vocab_size(), vocab_target.get_vocab_size()).to(device)


NameError: name 'build_transformer' is not defined

In [24]:
config = get_config()
file_path = str(Path('.').parent.resolve() / config["TRAIN_DIRECTORY"] / config["datasource"] / config["CHECKPOINT_DIRECTORY"] / config["model_name"])
print(file_path)


D:\Github\SUMMIT\src\train\opus_books\checkpoints\00


In [15]:
#Load pretrained weights
state = torch.load(file_path)

In [ ]:
model.load_state_dict(state['model_state_dict'])